# Лабораторная работа 2
## Задание:
1. Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

### Загрузка данных и подготовка датасета

In [14]:
dataSet = pd.read_csv('cybersecurity_attacks.csv')

label_encoder = LabelEncoder()
dataSet['Attack Type Encoded'] = label_encoder.fit_transform(dataSet['Attack Type'])

non_numeric_columns = dataSet.select_dtypes(exclude=['number']).columns.tolist()

dataSet = dataSet.drop(columns=non_numeric_columns)
dataSet.dropna(inplace=True)

### Разделение данных и нормализация

In [15]:
X = dataSet.drop('Attack Type Encoded', axis=1)
y = dataSet['Attack Type Encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Обучение и оценка качества на линейном ядре


In [ ]:
param_grid = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'C': [0.001, 0.01, 0.1, 1, 10],
              'degree': [1, 2, 3, 4, 5, 6, 7]}

svm_grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

svm_grid.fit(X_test, y_test)

best_params = svm_grid.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']
best_degree = best_params['degree']

print(f"Лучшие параметры: Ядро = {best_kernel}, C = {best_C}, Degree = {best_degree}")

### Подбор гиперпараметра с помощью перекрестной проверки

In [ ]:
svm_model = SVC(kernel='poly', degree=6, C=0.1)

# Обучение модели
svm_model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = svm_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print(f'Точность модели: {accuracy}')
print(report)

# Кросс-валидация
stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

